In [ ]:
import sys; sys.path.append('..')
import elastic_rods, numpy as np

In [ ]:
r = elastic_rods.ElasticRod([[x, 0, 0] for x in np.linspace(0, 1, 10)])
r.setMaterial(elastic_rods.RodMaterial('ellipse', 200, 0.3, [0.01, 0.005]))
pts = np.array(r.deformedPoints())
thetas = r.thetas()
pts += 1e-1 * np.random.random_sample(pts.shape)
r.setDeformedConfiguration(pts, thetas)

In [ ]:
from linkage_vis import LinkageViewer
view = LinkageViewer(r)
view.show()

Before we fixed the calculation of reference twists from the reference frame, reference twists were always computed in the range [-pi, pi]. So attempting to set a reference twist outside this range resulted in a different reference twist (corrupting the twisting energy, since thetas were adjusted assuming that the exact change requested was made):

In [ ]:
print(r.energyTwist())
dc = r.deformedConfiguration()
dc.setReferenceTwist(np.pi + 0.0000001)
dc.updateSourceFrame()
r.setDeformedConfiguration(pts, r.thetas())
print(r.energyTwist(), r.deformedConfiguration().referenceTwist[1])

When the reference twist crosses the bounds of this interval, the energy used to jump in a way that isn't predicted by the gradient. This can be tested for by setting the reference twist very close to the upper bound and running a finite difference validation.

In [ ]:
dc.setReferenceTwist(np.pi - 0.0000001 - 2 * np.pi)
dc.updateSourceFrame()
r.setDeformedConfiguration(pts, r.thetas())
print(r.energyTwist(), r.deformedConfiguration().referenceTwist[1])

In [ ]:
import finite_diff
finite_diff.fd_gradient_test(r, 1e-7, direction=np.random.random_sample(r.numDoF()))

In [ ]:
r.thetas()